In [25]:
import pandas as pd
import numpy as np
from functools import lru_cache
from pathlib import Path

def read_dat_files():
    R = pd.read_csv("ratings.csv",names=["userID","movieID","rating","timestamp"],skiprows=1)
    I = pd.read_csv("movies.csv",names=["movieID","title","genre"],skiprows=1)
    U = pd.read_csv("tags.csv",skiprows=1)
    return R, I, U 

R, I, U = read_dat_files()

In [26]:
R["rating"] = np.array(R["rating"],dtype=float) - 3.0

In [34]:
def create_movie_per_genre()->pd.DataFrame:
    if Path("./movies_genres.csv").isfile():
        return pd.read_csv("./movies_genres.csv",skiprows=1);
    R_I = pd.merge(R,I,how="inner",on="movieID")
    
    for i in R_I.rows:
        genre
    

userID
movieID
rating
timestamp
title
genre
